# train cars price model

Given the cars data that we have in snowflake

* Use xgboost and Scikit kearns random forest
* try MAPIE, for prediction intervals (`!pip install mapie`)

In [1]:
import os
from snowflake.snowpark import Session
from snowflake.snowpark import functions as F
from connection_config import connection_parameters

import pandas as pd

#### Current Environment Details
def current_snowflake_env():
    snowflake_environment = session.sql('select current_user(), current_role(), current_database(), current_schema(), current_version(), current_warehouse()').collect()
    print('User                     : {}'.format(snowflake_environment[0][0]))
    print('Role                     : {}'.format(snowflake_environment[0][1]))
    print('Database                 : {}'.format(snowflake_environment[0][2]))
    print('Schema                   : {}'.format(snowflake_environment[0][3]))
    print('Warehouse                : {}'.format(snowflake_environment[0][5]))
    print('Snowflake version        : {}'.format(snowflake_environment[0][4]))

#### Set up a connection with Snowflake
session = Session.builder.configs(connection_parameters).create()


In [2]:
current_snowflake_env()

User                     : SNOWFLAKETRIALUSER
Role                     : ACCOUNTADMIN
Database                 : SNOWFLAKE_SAMPLE_DATA
Schema                   : TPCH_SF10
Warehouse                : COMPUTE_WH
Snowflake version        : 7.12.5


In [3]:
cars_sf = session.table('CARS_DATA.PUBLIC.CAR_PRICES')

In [5]:
cars = cars_sf.to_pandas()

In [6]:
cars

,BOUWJAAR,KM_STAND,BRANDSTOF,MOTORINHOUD,VERMOGEN,TRANSMISSIE,TYPE,KLEUR,DEUR,PRIJS,MERK,MODEL,VRAAGPRIJS
0,2018,54700,Elektrisch,None,245kW,Automaat,Hatchback,Rood,5-deurs,â¬ 54.999,Tesla,Model,54999
1,2017,56266,Elektrisch,None,None,Automaat,Hatchback,Wit,5-deurs,â¬ 22.949,Volkswagen,e-Golf,22949
2,2021,1498,Elektrisch,None,None,Automaat,SUV / Terreinwagen,Groen,5-deurs,â¬ 38.745,Opel,Mokka,38745
3,2019,26805,Elektrisch,None,150kW,Automaat,SUV / Terreinwagen,Wit,5-deurs,â¬ 37.900,Hyundai,Kona,37900
4,2020,2785,Elektrisch,None,100kW,Automaat,Hatchback,Grijs,5-deurs,â¬ 20.495,Renault,Zoe,20495
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230995,1994,130521,Benzine,2799cc,142kW,Automaat,Sedan,Grijs,4-deurs,â¬ 12.950,Mercedes-Benz,S-klasse,12950
230996,2017,62676,Benzine,1595cc,115kW,Automaat,Sedan,Wit,4-deurs,â¬ 29.900,Mercedes-Benz,C-klasse,29900
230997,2017,76714,Benzine,1595cc,115kW,Automaat,Stationwagon,Zwart,5-deurs,â¬ 29.690,Mercedes-Benz,C-klasse,29690
230998,2015,103119,Diesel,2143cc,125kW,Automaat,Stationwagon,Grijs,5-deurs,â¬ 30.990,Mercedes-Benz,CLS-klasse,30990


In [ ]:
## create a random forest prediction model with the car data
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# split the data into training and test sets
train, test = train_test_split(cars, test_size=0.2)

# create a random forest model

rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=0)

# train the model
rf.fit(train.drop('price', axis=1), train['price'])
